In [ ]:
!pip install siibra siibra-jugex matplotlib nilearn

In [ ]:
import siibra
from nilearn import plotting
from IPython.display import display, Markdown

### Define two points in MNI space

We start by specifying some points in the reference space. Here, we use the MNI space. We can find such point, amongst other possibilities, by clicking them in the [siibra-explorer](https://siibra-explorer.apps.hbp.eu/).

In [ ]:
import siibra
from nilearn import plotting

pt_mni = siibra.Point(
    (-25.65, -2.75, -33.75), space="mni152")
view1 = plotting.plot_anat(
    cut_coords=tuple(pt_mni), 
    title="Point in MNI152 space")
view1.add_markers([tuple(pt_mni)])

pt_bb = pt_mni.warp('bigbrain')
atlas = siibra.atlases['human']
bigbrain = atlas.get_template("bigbrain")
view2 = plotting.plot_anat(
    bigbrain.fetch(), cut_coords=tuple(pt_bb), 
    title="Point in BigBrain space")
view2.add_markers([tuple(pt_bb)])

In [ ]:
import siibra
from nilearn import plotting
point = siibra.Point((-25.65, -2.75, -33.75), space="mni152")
pmaps = siibra.atlases['human'].get_map(
    space="mni152", parcellation="julich 2.9", maptype="continuous")
region, pmap, scores = pmaps.assign_coordinates(point, sigma_mm=5)[0]
view = plotting.plot_stat_map( 
    pmap, cut_coords=tuple(point),
    title=f"{region.name} ({scores['correlation']:.2f})") 
view.add_markers([tuple(point)],['cyan'])

In [ ]:
# These are copy-pasted from the interactive atlas viewer:
points = siibra.PointSet(
    [
        "-25.650mm, -2.750mm, -33.750mm",
        "-37.350mm, -81.050mm, -6.300mm"
    ], 
    space='mni152')

In [ ]:
plotting.view_markers(list(map(tuple,points)), ['red', 'cyan'], marker_size=10)  

### Assign brain regions to the 3D points

We assign the points to brain regions from the Julich-Brain cytoarchitectonic atlas, using a certain location tolerance of 5mm standard deviation.

In [ ]:
atlas = siibra.atlases['human']
pmaps = atlas.get_map(space="mni152",parcellation="julich",maptype="continuous")
regions1, regions2 = pmaps.assign_coordinates(points,sigma_mm=5)

Let's look at the most probable area found for each point.

In [ ]:
region1,statmap1,scores1 = regions1[0]
region2,statmap2,scores2 = regions2[0]
view1 = plotting.plot_stat_map(
    statmap1, title=f"{region1.name} ({scores1['correlation']:.2f})", cut_coords=tuple(points[0]))
view2 = plotting.plot_stat_map(
    statmap2, title=f"{region2.name} ({scores2['correlation']:.2f})", cut_coords=tuple(points[1]))
view1.add_markers([tuple(points[0])],['cyan'])
view2.add_markers([tuple(points[1])],['cyan'])

### Look for white matter bundles connecting the regions

Next, we look at the most probable region associated to each point, considering them as a source and target region to investigate connectivity. In order to find white matter fibre bundles which are likely to connect them, we use the probability maps of the white matter fibre bundle parcellation, and assign both the source and target region from Julich-Brain to the fibre bundles. The intersection of the resulting bundles gives us those that are likely to provide connections from source to target.

In [ ]:
# get the probabilistic maps of long fibre bundles
bundlemaps = atlas.get_map("mni152","long bundles","continuous")

# assign the source and target region to the bundles, 
# using their probability maps found above.
bundles1 = bundlemaps.assign(statmap1,msg=f"Find bundles touching {region1.name}")
bundles2 = bundlemaps.assign(statmap2,msg=f"Find bundles touching {region2.name}")
                
# intersect the two sets of related fiber bundles
intersection = {
    (b[0],b[1]) for b in bundles1+bundles2
    if  any(b1[0]==b[0] for b1 in bundles1) 
    and any(b2[0]==b[0] for b2 in bundles2) }

In [ ]:
# plot each of the bundles in the intersection (but no more than 3)
for bundle,bundlemap in intersection:
    view=plotting.plot_stat_map(bundlemap,
                                title=f"{region1.name} and {region2.name} connected by {bundle.name} ")
    view.add_overlay(statmap1)
    view.add_overlay(statmap2)
    view.add_markers([tuple(points[0]),tuple(points[1])],['white'])

### Find connectivity in terms of streamlines from DTI

We also investigate the connectivtiy of the two regions as measured by in-vivo imaging. To do so, we select the first region in the atlas, and search for connectivity profiles.

In [ ]:
# select the source region
with siibra.QUIET:
    p, = siibra.get_features(region1,"ConnectivityProfile")
print(p.name)
#display(Markdown(p.description))

We use a parcellation object to decode the names of the connected regions.

In [ ]:
with siibra.QUIET:
    connections = p.decode(atlas.get_parcellation("julich"))

Create a bar plot of the connectivity profile, and identify the target region from above.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

i = [r for v,r in connections].index(region2)

N = max(20,i)
X = range(N)
Y = [v for v,_ in connections[:N]]
L = [r.name for v,r in connections[:N]]

fig  = plt.figure(figsize=(12,4))
ax1  = fig.add_subplot()
ax1.set_xticks(X)
ax1.set_xticklabels(L,rotation=60,ha='right')
ax1.bar(X,Y,0.8)

# where is region2?
i = [r for v,r in connections].index(region2)
ax1.bar(i,connections[i][0],0.9,color='r')

fig.gca().set_title(f"Connection strengths from area {region1.name}")
plt.show()